# Lab 09: Advanced DALL-E Image Generation

## Overview
This advanced notebook explores sophisticated image generation techniques with Azure OpenAI's DALL-E. Learn prompt engineering best practices, style transfer, creative workflows, and production-ready patterns.

## Advanced Topics Covered
- Prompt engineering best practices and patterns
- Style transfer and artistic effects
- Image variations and iterative refinement
- Batch generation with parameters
- Quality optimization techniques
- Content policy and safety considerations
- Creative workflows and use cases
- A/B testing prompts and results

## Setup

In [ ]:
!pip install openai python-dotenv pillow requests matplotlib -q

In [ ]:
import os
import json
import requests
import time
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Tuple
from dotenv import load_dotenv
from openai import AzureOpenAI
from PIL import Image
from IPython.display import display, HTML, Markdown
import matplotlib.pyplot as plt

# Load configuration
load_dotenv('python/.env')
endpoint = os.getenv("ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT")
api_version = os.getenv("API_VERSION")

# Initialize client
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

print("✓ Environment initialized")

## 1. Advanced Prompt Engineering Patterns

Master the art of prompt construction with proven patterns.

In [ ]:
class PromptBuilder:
    """Build structured prompts with modular components."""
    
    def __init__(self):
        self.components = {
            'subject': '',
            'style': '',
            'composition': '',
            'lighting': '',
            'color': '',
            'mood': '',
            'technical': '',
            'additional': ''
        }
    
    def set_subject(self, subject: str):
        self.components['subject'] = subject
        return self
    
    def set_style(self, style: str):
        self.components['style'] = style
        return self
    
    def set_composition(self, composition: str):
        self.components['composition'] = composition
        return self
    
    def set_lighting(self, lighting: str):
        self.components['lighting'] = lighting
        return self
    
    def set_color(self, color: str):
        self.components['color'] = color
        return self
    
    def set_mood(self, mood: str):
        self.components['mood'] = mood
        return self
    
    def set_technical(self, technical: str):
        self.components['technical'] = technical
        return self
    
    def add_detail(self, detail: str):
        self.components['additional'] += f", {detail}"
        return self
    
    def build(self) -> str:
        """Construct the final prompt from components."""
        parts = [v for v in self.components.values() if v]
        return ', '.join(parts).strip(', ')

# Example: Build a complex prompt programmatically
prompt = (PromptBuilder()
    .set_subject("a basket of exotic tropical fruits")
    .set_style("professional product photography")
    .set_composition("centered on marble countertop, rule of thirds")
    .set_lighting("soft natural window light from left, golden hour")
    .set_color("vibrant saturated colors, warm tones")
    .set_mood("fresh, appetizing, premium quality")
    .set_technical("shot with 85mm lens, f/2.8, shallow depth of field")
    .add_detail("high resolution")
    .add_detail("professional food styling")
    .build())

print("🏗️ Constructed Prompt:\n")
print(prompt)
print("\n" + "="*80 + "\n")

# Generate with the constructed prompt
result = client.images.generate(
    model=model_deployment,
    prompt=prompt,
    size="1024x1024",
    quality="hd"
)

# Display
img_url = result.data[0].url
img = Image.open(requests.get(img_url, stream=True).raw)
display(img.resize((512, 512)))

## 2. Style Transfer and Artistic Effects

Generate images in various artistic styles with precision.

In [ ]:
# Define artistic style templates
STYLE_TEMPLATES = {
    'impressionist': "in the style of Claude Monet, impressionist painting with visible brushstrokes, {subject}",
    'art_nouveau': "Art Nouveau poster style with flowing organic lines and decorative borders, {subject}",
    'ukiyo-e': "Japanese ukiyo-e woodblock print style with flat colors and bold outlines, {subject}",
    'cubist': "Cubist painting in the style of Picasso, geometric shapes and multiple perspectives, {subject}",
    'pop_art': "Pop art style like Andy Warhol, bold colors and high contrast, {subject}",
    'art_deco': "Art Deco style with geometric patterns and metallic accents, {subject}",
    'surrealist': "Surrealist painting in the style of Salvador Dali, dreamlike and fantastical, {subject}",
    'minimalist': "Minimalist modern design, clean lines and simple forms, {subject}"
}

def generate_style_variations(subject: str, styles: List[str]):
    """Generate the same subject in multiple artistic styles."""
    results = []
    
    for style in styles:
        if style in STYLE_TEMPLATES:
            prompt = STYLE_TEMPLATES[style].format(subject=subject)
            print(f"\n🎨 Style: {style.upper()}")
            print(f"Prompt: {prompt}\n")
            
            result = client.images.generate(
                model=model_deployment,
                prompt=prompt,
                size="1024x1024",
                quality="standard"
            )
            
            img_url = result.data[0].url
            img = Image.open(requests.get(img_url, stream=True).raw)
            display(img.resize((400, 400)))
            
            results.append({
                'style': style,
                'url': img_url,
                'prompt': prompt
            })
            
            time.sleep(1)  # Rate limiting
    
    return results

# Generate style variations
subject = "a bowl of fresh mangoes and oranges on a table"
styles_to_try = ['impressionist', 'pop_art', 'ukiyo-e', 'minimalist']

print("🎭 Generating Style Variations...\n")
print("="*80)
style_results = generate_style_variations(subject, styles_to_try)

## 3. Prompt A/B Testing

Compare different prompt variations to find the best approach.

In [ ]:
def ab_test_prompts(prompt_variants: Dict[str, str], size="1024x1024"):
    """Generate and compare multiple prompt variations."""
    
    results = {}
    
    for name, prompt in prompt_variants.items():
        print(f"\n{'='*80}")
        print(f"Variant: {name}")
        print('='*80)
        print(f"Prompt: {prompt}\n")
        
        result = client.images.generate(
            model=model_deployment,
            prompt=prompt,
            size=size,
            quality="standard"
        )
        
        img_url = result.data[0].url
        img = Image.open(requests.get(img_url, stream=True).raw)
        display(img.resize((400, 400)))
        
        results[name] = {
            'prompt': prompt,
            'url': img_url
        }
        
        time.sleep(1)
    
    return results

# Test different prompt strategies
test_variants = {
    'Simple': "tropical fruits",
    'Descriptive': "fresh colorful tropical fruits including mango, papaya, and dragon fruit",
    'Detailed': "a professional photograph of exotic tropical fruits arranged artistically, vibrant colors, natural lighting",
    'Ultra-Detailed': "a studio product photograph of premium tropical fruits including ripe mango, fresh papaya, and dragon fruit, arranged on a rustic wooden board, soft diffused lighting from above, shallow depth of field, food photography, high resolution, appetizing presentation"
}

print("🧪 A/B Testing Prompt Variants\n")
ab_results = ab_test_prompts(test_variants)

## 4. Batch Generation with Parameter Grid

Systematically explore parameter combinations.

In [ ]:
def parameter_grid_search(base_prompt: str, param_grid: Dict):
    """Generate images with different parameter combinations."""
    
    results = []
    
    for size in param_grid.get('sizes', ['1024x1024']):
        for quality in param_grid.get('qualities', ['standard']):
            for style_mod in param_grid.get('style_modifiers', ['']):
                prompt = f"{base_prompt}{', ' + style_mod if style_mod else ''}"
                
                print(f"\nGenerating: {size}, {quality}, {style_mod or 'base'}")
                
                try:
                    result = client.images.generate(
                        model=model_deployment,
                        prompt=prompt,
                        size=size,
                        quality=quality
                    )
                    
                    img_url = result.data[0].url
                    
                    results.append({
                        'prompt': prompt,
                        'size': size,
                        'quality': quality,
                        'style_modifier': style_mod,
                        'url': img_url,
                        'status': 'success'
                    })
                    
                    # Display
                    img = Image.open(requests.get(img_url, stream=True).raw)
                    display_width = 300 if size == "1024x1024" else 400
                    aspect = img.height / img.width
                    display(img.resize((display_width, int(display_width * aspect))))
                    
                except Exception as e:
                    print(f"Error: {e}")
                    results.append({
                        'prompt': prompt,
                        'size': size,
                        'quality': quality,
                        'error': str(e),
                        'status': 'failed'
                    })
                
                time.sleep(1)
    
    return results

# Define parameter grid
param_grid = {
    'sizes': ['1024x1024', '1792x1024'],
    'qualities': ['standard', 'hd'],
    'style_modifiers': [
        '',
        'vibrant colors',
        'soft pastel tones'
    ]
}

base_prompt = "a minimalist illustration of a mango"

print("🔬 Parameter Grid Search\n")
print("="*80)
grid_results = parameter_grid_search(base_prompt, param_grid)

# Summary
print("\n" + "="*80)
print(f"Generated {len([r for r in grid_results if r['status'] == 'success'])} images")
print(f"Failed: {len([r for r in grid_results if r['status'] == 'failed'])}")

## 5. Quality Optimization Techniques

Strategies for consistently high-quality results.

In [ ]:
QUALITY_ENHANCERS = {
    'photography': [
        "professional photography",
        "high resolution",
        "sharp focus",
        "professional lighting",
        "DSLR quality"
    ],
    'art': [
        "masterpiece",
        "highly detailed",
        "professional art",
        "award winning",
        "trending on artstation"
    ],
    'commercial': [
        "commercial quality",
        "professional studio",
        "advertising photography",
        "premium quality",
        "magazine quality"
    ]
}

def enhance_prompt_quality(base_prompt: str, category: str = 'photography') -> str:
    """Add quality enhancers to a prompt."""
    enhancers = QUALITY_ENHANCERS.get(category, QUALITY_ENHANCERS['photography'])
    enhanced = f"{base_prompt}, {', '.join(enhancers)}"
    return enhanced

# Compare base vs enhanced prompts
base = "a fruit bowl on a table"

print("📈 Quality Enhancement Comparison\n")
print("="*80)

# Base prompt
print("\n1. BASE PROMPT")
print(f"Prompt: {base}\n")
result1 = client.images.generate(
    model=model_deployment,
    prompt=base,
    size="1024x1024"
)
img1 = Image.open(requests.get(result1.data[0].url, stream=True).raw)
display(img1.resize((400, 400)))

time.sleep(1)

# Enhanced prompt
enhanced = enhance_prompt_quality(base, 'photography')
print("\n2. ENHANCED PROMPT")
print(f"Prompt: {enhanced}\n")
result2 = client.images.generate(
    model=model_deployment,
    prompt=enhanced,
    size="1024x1024",
    quality="hd"
)
img2 = Image.open(requests.get(result2.data[0].url, stream=True).raw)
display(img2.resize((400, 400)))

## 6. Creative Workflows

End-to-end workflows for specific use cases.

In [ ]:
class CreativeWorkflow:
    """Workflow manager for creative image generation projects."""
    
    def __init__(self, project_name: str):
        self.project_name = project_name
        self.project_dir = Path("images") / project_name
        self.project_dir.mkdir(parents=True, exist_ok=True)
        self.generations = []
    
    def generate_concept_variations(self, base_concept: str, variations: List[str]):
        """Generate multiple variations of a concept."""
        print(f"\n🎨 Project: {self.project_name}")
        print(f"📁 Output: {self.project_dir}\n")
        print("="*80)
        
        for i, variation in enumerate(variations, 1):
            prompt = f"{base_concept}, {variation}"
            print(f"\n[{i}/{len(variations)}] {variation}")
            
            result = client.images.generate(
                model=model_deployment,
                prompt=prompt,
                size="1024x1024",
                quality="standard"
            )
            
            img_url = result.data[0].url
            
            # Save
            filename = f"{self.project_name}_v{i:02d}.png"
            filepath = self.project_dir / filename
            img_data = requests.get(img_url).content
            with open(filepath, 'wb') as f:
                f.write(img_data)
            
            self.generations.append({
                'variation': variation,
                'prompt': prompt,
                'file': str(filepath),
                'url': img_url
            })
            
            # Display
            img = Image.open(filepath)
            display(img.resize((350, 350)))
            
            time.sleep(1)
        
        self.save_metadata()
        return self.generations
    
    def save_metadata(self):
        """Save project metadata."""
        metadata = {
            'project': self.project_name,
            'created': datetime.now().isoformat(),
            'generations': self.generations
        }
        metadata_file = self.project_dir / 'metadata.json'
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)
        print(f"\n✓ Metadata saved: {metadata_file}")

# Example: Social media campaign workflow
workflow = CreativeWorkflow("fruit_campaign_2024")

base_concept = "fresh tropical fruits, modern advertising style"
variations = [
    "bright and energetic, for young audience",
    "elegant and sophisticated, luxury market",
    "natural and organic, health-focused",
    "fun and playful, family-friendly"
]

campaign_images = workflow.generate_concept_variations(base_concept, variations)

print("\n" + "="*80)
print(f"✓ Campaign complete: {len(campaign_images)} variations generated")

## 7. Content Safety and Moderation

Best practices for responsible AI image generation.

In [ ]:
def safe_generate(prompt: str, safety_checks: Dict = None) -> Dict:
    """
    Generate images with safety considerations.
    
    Safety checks:
    - Prompt validation
    - Content policy compliance
    - Error handling
    - Logging
    """
    
    # Default safety settings
    if safety_checks is None:
        safety_checks = {
            'max_prompt_length': 1000,
            'log_prompts': True,
            'retry_on_filter': False
        }
    
    # Validate prompt length
    if len(prompt) > safety_checks.get('max_prompt_length', 1000):
        return {
            'status': 'error',
            'message': 'Prompt too long',
            'prompt_length': len(prompt)
        }
    
    # Log if enabled
    if safety_checks.get('log_prompts'):
        print(f"[SAFE_GEN] Prompt: {prompt[:100]}...")
    
    try:
        result = client.images.generate(
            model=model_deployment,
            prompt=prompt,
            size="1024x1024"
        )
        
        return {
            'status': 'success',
            'url': result.data[0].url,
            'prompt': prompt
        }
        
    except Exception as e:
        error_msg = str(e)
        
        # Check if content filtered
        if 'content_policy' in error_msg.lower() or 'filtered' in error_msg.lower():
            return {
                'status': 'filtered',
                'message': 'Content policy violation',
                'error': error_msg
            }
        
        return {
            'status': 'error',
            'message': 'Generation failed',
            'error': error_msg
        }

# Test safe generation
print("🛡️ Safe Generation Example\n")
safe_prompt = "a professional photograph of fresh organic fruits"
result = safe_generate(safe_prompt)

if result['status'] == 'success':
    print(f"\n✓ Generation successful")
    img = Image.open(requests.get(result['url'], stream=True).raw)
    display(img.resize((400, 400)))
else:
    print(f"\n⚠️  Status: {result['status']}")
    print(f"Message: {result.get('message', 'Unknown error')}")

## 8. Advanced Prompt Library

A collection of proven prompts for various scenarios.

In [ ]:
PROMPT_LIBRARY = {
    'product_photography': {
        'template': "{product}, professional product photography, {background}, {lighting}, high resolution, commercial quality, {angle}",
        'variables': {
            'background': ['white background', 'marble surface', 'wooden table', 'lifestyle setting'],
            'lighting': ['studio lighting', 'natural light', 'dramatic lighting', 'soft diffused light'],
            'angle': ['top view', 'front view', '45 degree angle', 'close-up']
        }
    },
    'marketing': {
        'template': "{subject}, {style}, {mood}, perfect for {purpose}, {composition}",
        'variables': {
            'style': ['modern', 'minimalist', 'bold and vibrant', 'elegant'],
            'mood': ['energetic', 'calm and peaceful', 'exciting', 'professional'],
            'purpose': ['social media', 'print advertisement', 'website banner', 'packaging'],
            'composition': ['centered', 'asymmetric', 'lots of negative space', 'busy composition']
        }
    },
    'artistic': {
        'template': "{subject}, {art_style}, {technique}, {color_scheme}, {inspiration}",
        'variables': {
            'art_style': ['impressionist', 'abstract', 'surrealist', 'pop art', 'art nouveau'],
            'technique': ['oil painting', 'watercolor', 'digital art', 'mixed media'],
            'color_scheme': ['vibrant colors', 'pastel tones', 'monochromatic', 'complementary colors'],
            'inspiration': ['inspired by nature', 'geometric patterns', 'organic forms', 'cultural motifs']
        }
    }
}

def use_prompt_template(category: str, subject: str, custom_vars: Dict = None):
    """Generate a prompt from a template."""
    
    if category not in PROMPT_LIBRARY:
        return None
    
    template_data = PROMPT_LIBRARY[category]
    template = template_data['template']
    variables = template_data['variables']
    
    # Use custom variables or pick random defaults
    import random
    
    if custom_vars is None:
        custom_vars = {}
    
    for var_name, options in variables.items():
        if var_name not in custom_vars:
            custom_vars[var_name] = random.choice(options)
    
    custom_vars['subject'] = subject
    custom_vars['product'] = subject
    
    prompt = template.format(**custom_vars)
    return prompt

# Example: Use templates
print("📚 Prompt Library Examples\n")
print("="*80)

categories = ['product_photography', 'marketing', 'artistic']

for category in categories:
    print(f"\n📖 Category: {category.upper()}")
    prompt = use_prompt_template(category, "fresh tropical mango")
    print(f"Generated prompt:\n{prompt}\n")
    print("-" * 80)

## 9. Performance Optimization

Strategies for efficient batch generation and cost optimization.

In [ ]:
import time
from datetime import datetime

class OptimizedGenerator:
    """Optimized image generation with rate limiting and caching."""
    
    def __init__(self, rate_limit_delay=1.0):
        self.rate_limit_delay = rate_limit_delay
        self.last_request_time = 0
        self.cache = {}
        self.metrics = {
            'total_requests': 0,
            'cache_hits': 0,
            'errors': 0,
            'total_time': 0
        }
    
    def _rate_limit(self):
        """Enforce rate limiting."""
        elapsed = time.time() - self.last_request_time
        if elapsed < self.rate_limit_delay:
            time.sleep(self.rate_limit_delay - elapsed)
        self.last_request_time = time.time()
    
    def generate(self, prompt: str, use_cache=True, **kwargs):
        """Generate with optimizations."""
        
        # Check cache
        cache_key = f"{prompt}_{kwargs.get('size', '1024x1024')}_{kwargs.get('quality', 'standard')}"
        
        if use_cache and cache_key in self.cache:
            print("✓ Cache hit")
            self.metrics['cache_hits'] += 1
            return self.cache[cache_key]
        
        # Rate limit
        self._rate_limit()
        
        # Generate
        start_time = time.time()
        self.metrics['total_requests'] += 1
        
        try:
            result = client.images.generate(
                model=model_deployment,
                prompt=prompt,
                size=kwargs.get('size', '1024x1024'),
                quality=kwargs.get('quality', 'standard')
            )
            
            generation_time = time.time() - start_time
            self.metrics['total_time'] += generation_time
            
            result_data = {
                'url': result.data[0].url,
                'prompt': prompt,
                'generation_time': generation_time
            }
            
            # Cache result
            if use_cache:
                self.cache[cache_key] = result_data
            
            return result_data
            
        except Exception as e:
            self.metrics['errors'] += 1
            raise e
    
    def get_metrics(self):
        """Get performance metrics."""
        return {
            **self.metrics,
            'avg_time': self.metrics['total_time'] / max(self.metrics['total_requests'], 1),
            'cache_hit_rate': self.metrics['cache_hits'] / max(self.metrics['total_requests'] + self.metrics['cache_hits'], 1)
        }

# Test optimized generator
print("⚡ Performance Optimization Test\n")
gen = OptimizedGenerator(rate_limit_delay=0.5)

prompts = [
    "a simple mango illustration",
    "a simple mango illustration",  # Duplicate for cache test
    "an orange fruit photograph",
    "a simple mango illustration"  # Another duplicate
]

for i, prompt in enumerate(prompts, 1):
    print(f"\n[{i}/{len(prompts)}] {prompt}")
    result = gen.generate(prompt)
    print(f"Generation time: {result['generation_time']:.2f}s")

# Show metrics
print("\n" + "="*80)
print("📊 Performance Metrics")
print("="*80)
metrics = gen.get_metrics()
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

## Summary

In this advanced lab, you explored:

✅ **Prompt engineering patterns** - Structured, modular prompt construction  
✅ **Style transfer** - Artistic style variations and effects  
✅ **A/B testing** - Systematic prompt comparison  
✅ **Parameter optimization** - Grid search and quality enhancement  
✅ **Creative workflows** - End-to-end project management  
✅ **Content safety** - Responsible AI practices  
✅ **Prompt libraries** - Reusable templates and patterns  
✅ **Performance optimization** - Caching, rate limiting, metrics  

## Best Practices

### Prompt Engineering
- Use structured, modular prompts
- Include style, mood, and technical details
- Add quality enhancers for better results
- Test variations systematically

### Production Considerations
- Implement rate limiting
- Cache repeated generations
- Log all generations for audit
- Handle errors gracefully
- Monitor costs and usage

### Quality Optimization
- Use HD quality for commercial work
- Test different aspect ratios
- A/B test prompts before scaling
- Build prompt libraries from successful patterns

### Content Safety
- Validate prompts before generation
- Handle content policy violations
- Implement human review for sensitive applications
- Follow Azure OpenAI responsible AI guidelines

## Advanced Use Cases

- **Marketing campaigns**: Generate branded visual content at scale
- **Product visualization**: Create mockups and concept art
- **Content creation**: Social media, blogs, and articles
- **Design iteration**: Rapid prototyping and A/B testing
- **Personalization**: User-specific visual content
- **Educational materials**: Custom illustrations and diagrams